# Extracting events from TCRM database

This notebook provides an interactive way of interrogating the event database generated by TCRM. You can select a location, then extract all events that pass within a defined distance of the location. The tracks will be plotted, and you can select a single track from all the events.

We start with the usual process of importing the required modules.

In [ ]:
%matplotlib notebook
from __future__ import print_function, division

In [ ]:
import os
import io
import sys

import database
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

import matplotlib.cm as cm
import pandas as pd


from Utilities.config import ConfigParser
from Utilities.track import ncReadTrackData


# Import widgets for interactive notebook
from ipywidgets import interact, fixed, Dropdown, FloatSlider, interact_manual, interactive
import ipywidgets as widgets

import seaborn as sns
sns.set_context("poster")
sns.set_style("whitegrid")

Set up a configuration string - make sure to check the paths are correct!

In [ ]:
configstr = """
[DataProcess]
InputFile=C:/WorkSpace/data/Allstorms.ibtracs_wmo.v03r09.csv
Source=IBTRACS
StartSeason=1981
FilterSeasons=False

[Region]
; Domain for windfield and hazard calculation
gridLimit={'xMin':175.,'xMax':185.,'yMin':-23.0,'yMax':-13.0}
gridSpace={'x':1.0,'y':1.0}
gridInc={'x':1.0,'y':0.5}

[TrackGenerator]
NumSimulations=10000
YearsPerSimulation=1
SeasonSeed=68876543
TrackSeed=334825
TimeStep=1.0

[Input]
landmask = C:/WorkSpace/tcrm/input/landmask.nc
mslpfile = C:/WorkSpace/tcrm/MSLP/slp.day.ltm.nc
datasets = IBTRACS,LTMSLP

[Output]
Path=C:/workSpace/data/share/tcrm/fiji

[Hazard]
Years=2,5,10,20,25,50,100,200,250,500,1000,2000,2500,5000
MinimumRecords=10
CalculateCI=False

[Logging]
LogFile=C:/WorkSpace/data/share/tcrm/fiji/log/fiji.log
LogLevel=INFO
Verbose=False

[IBTRACS]
; Input data file settings
url = ftp://eclipse.ncdc.noaa.gov/pub/ibtracs/v03r06/wmo/csv/Allstorms.ibtracs_wmo.v03r09.csv.gz
path = C:/WorkSpace/tcrm/input/
filename = Allstorms.ibtracs_wmo.v03r09.csv
columns = tcserialno,season,num,skip,skip,skip,date,skip,lat,lon,skip,pressure
fielddelimiter = ,
numberofheadinglines = 3
pressureunits = hPa
lengthunits = km
dateformat = %Y-%m-%d %H:%M:%S
speedunits = kph

[LTMSLP]
; MSLP climatology file settings
URL = ftp://ftp.cdc.noaa.gov/Datasets/ncep.reanalysis.derived/surface/slp.day.1981-2010.ltm.nc
path = C:/WorkSpace/tcrm/MSLP
filename = slp.day.ltm.nc

[Process]
DatFile=C:/WorkSpace/data/share/tcrm/fiji/process/dat/fiji.dat
ExcludePastProcessed=False
"""

Now we read the configuration string into a `ConfigParser` instance, and load up the simulation database. We get a list of all the locations in the simulation domain as well, so we can choose a location for extracting tracks.

In [ ]:
config = ConfigParser()
config.readfp(io.BytesIO(configstr))

db = database.HazardDatabase(configstr)
locations = db.getLocations()
locNameList = list(locations['locName'])
outputPath = config.get('Output', 'Path')

These next three functions help to select the tracks that pass within a given distance of a selected location, and to load the tracks from the netCDF files that hold the tracks.

In [ ]:
def loadTrack(trackId):
    trackNum, trackYear = int(trackId.split('-')[0]), int(trackId.split('-')[1])
    trackFile = os.path.join(outputPath, 'tracks', 'tracks.{0:05d}.nc'.format(trackYear))
    tracks = ncReadTrackData(trackFile)
    
    return [t for t in tracks if t.trackId==(trackNum, trackYear)][0]

def getTracks(recs):
    tracks = []
    for rec in recs:
        trackId = rec['eventId']
        track = loadTrack(trackId)
        tracks.append(track)
    return tracks

def selectLocTracks(locName, distance):
    locId = locations['locId'][locations['locName']==locName][0]
    locLon = locations['locLon'][locations['locId']==locId][0]
    locLat = locations['locLat'][locations['locId']==locId][0]
    recs = database.locationPassageWindSpeed(db, locId, 30, distance)
    tracks = getTracks(recs)
    
    return tracks, locId, distance

The plotting routine. Here we create a `Basemap` instance to display a map, then cycle through the list of tracks passed in, plotting each in a grey line. There's also a `trackId` variable. That track will be plotted in red to stand out from the crowd, and you can save it at a later point...

In [ ]:
def plotTracks(tracks, trackId, locId, distance):
    trackNum, trackYear = trackId.split('-')
    locName = locations['locName'][locations['locId']==locId][0]
    locLon = locations['locLon'][locations['locId']==locId][0]
    locLat = locations['locLat'][locations['locId']==locId][0]
    domain = config.geteval('Region', 'gridLimit')
    xx = np.arange(domain['xMin'], domain['xMax'] + 0.1, 0.1)
    yy = np.arange(domain['yMin'], domain['yMax'] + 0.1, 0.1)
    [xgrid, ygrid] = np.meshgrid(xx,yy)
    fig, axes = plt.subplots(figsize=(8, 8))
    mapkwargs = dict(llcrnrlon=domain['xMin'],
                     llcrnrlat=domain['yMin'],
                     urcrnrlon=domain['xMax'],
                     urcrnrlat=domain['yMax'],
                     resolution='f',
                     projection='merc')
    mapobj = Basemap(ax=axes, **mapkwargs)
    mx, my = mapobj(xgrid, ygrid)
    xmin = mapobj.llcrnrlon
    xmax = mapobj.urcrnrlon
    ymin = mapobj.llcrnrlat
    ymax = mapobj.urcrnrlat

    dx = abs(xmin - xmax)
    dy = abs(ymin - ymax)
    dd = max(dx, dy)
    gr_opts = np.array([30., 10., 5., 4., 2.])
    min_gr = 5
    try:
        dl = gr_opts[np.where((dd/gr_opts) >= min_gr)[0][0]]
    except IndexError:
        dl = 2.

    meridians = np.arange(dl*np.floor(xmin / dl),
                          dl*np.ceil(xmax / dl) + dl, dl)
    parallels = np.arange(dl*np.floor(ymin / dl),
                          dl*np.ceil(ymax / dl) + dl, dl)

    mapobj.drawparallels(parallels, linewidth=0.25,
                         labels=[1, 0, 0, 1], style="italic")
    mapobj.drawmeridians(meridians, linewidth=0.25,
                         labels=[1, 0, 0, 1], style='italic')

    mapobj.fillcontinents(color="#F4B342",
                          lake_color="#BEE8FF",
                          zorder=0)
    mapobj.drawcoastlines(linewidth=.5, color="k")
    mapobj.drawmapboundary(fill_color="#BEE8FF")
    for t in tracks:
        mlon, mlat = mapobj(t.Longitude, t.Latitude)
        mlocLon, mlocLat = mapobj(locLon, locLat)
        if t.trackId == (int(trackId.split('-')[0]), int(trackId.split('-')[1])):
            selTrack = t
            mapobj.plot(mlon, mlat, linewidth=2, color='r')
        else:
            mapobj.plot(mlon, mlat, linewidth=1, color='0.5')
        mapobj.plot(mlocLon, mlocLat, marker='o', color='k', markersize=5)

    axes.set_title("TCs passing within {0} km of {1}".format(int(distance), locName))
    fig.tight_layout()
    plt.show()
    return selTrack

In [ ]:
w = interactive(selectLocTracks, 
            locName=Dropdown(options=locNameList, description="Location"),
            distance=FloatSlider(value=50, min=5, max=100, step=1.0, 
                                 description="Closest passage", 
                                 continuous_update=False)
               )
display(w)

Each time you change the distance or the location above, you'll need to re-run this next cell, since it relies on updating the result from above. 

Now you can select a track from the dropdown list, and it'll be plotted in red (compared to the others all plotted in grey).

In [ ]:
tracklist, locId, distance = w.result
if len(tracklist) == 0:
    print("No tracks available")
else:
    trackIdList = ["{0}-{1}".format(t.trackId[0], t.trackId[1]) for t in tracklist]
    x = interactive(plotTracks, tracks=fixed(tracklist),
                    trackId=Dropdown(options=trackIdList, description="Track ID", value=trackIdList[0]),
                    locId=fixed(locId), distance=fixed(distance))
    display(x)

Once you have found a track that looks a likely candidate for further simulation, you can execute the next cell, which will save the track to a TCRM-format track file. This track file will be initially stored in the main output folder for TCRM (what you've configured above), but I recommend moving it to another location 

In [ ]:
from Utilities.track import ncSaveTracks
scenarioTrackFile = os.path.join(outputPath, 'track.{0:03d}-{1:05d}.nc'.format(*x.result.trackId))
print(scenarioTrackFile)
ncSaveTracks(scenarioTrackFile, [x.result])